In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
#讀取.txt資料到array
array = []
f = open('auto-mpg.data.txt')
data = f.readlines()
for i in range(len(data)):
    array_list = []
    for j in data[i].split():
        array_list.append(j)
    array.append(array_list)        
#print(array)

In [3]:
#另存car name欄位資料
car_list = []
for i in data:
    i = i.split("\"")
    car_list.append(i[1])
#print(car_list)

In [4]:
#DataFrame
car = pd.DataFrame(car_list, columns= ['car name'])
#car

In [5]:
#DataFrame
df1 = pd.DataFrame(array)
del_col = [8,9,10,11,12,13]
df1.drop(df1.columns[del_col], axis=1, inplace=True)
#df1

In [6]:
#合併所有資料、rename
df = pd.concat([car,df1],axis=1)
df.set_index('car name', inplace=True)
df = df.set_axis(['mpg', 'cylinders', 'displacement', 'horsepower', 
                  'weight', 'acceleration', 'model year', 'origin'],
                 axis=1, inplace=False)
df

mpg cylinders displacement horsepower weight  \
car name                                                                   
chevrolet chevelle malibu  18.0         8        307.0      130.0  3504.   
buick skylark 320          15.0         8        350.0      165.0  3693.   
plymouth satellite         18.0         8        318.0      150.0  3436.   
amc rebel sst              16.0         8        304.0      150.0  3433.   
ford torino                17.0         8        302.0      140.0  3449.   
...                         ...       ...          ...        ...    ...   
ford mustang gl            27.0         4        140.0      86.00  2790.   
vw pickup                  44.0         4        97.00      52.00  2130.   
dodge rampage              32.0         4        135.0      84.00  2295.   
ford ranger                28.0         4        120.0      79.00  2625.   
chevy s-10                 31.0         4        119.0      82.00  2720.   

                          acceleration model year origin  
car name                                                  
chevrolet chevelle malibu         12.0         70      1  
buick skylark 320                 11.5         70      1  
plymouth satellite                11.0         70      1  
amc rebel sst                     12.0         70      1  
ford torino                       10.5         70      1  
...                                ...        ...    ...  
ford mustang gl                   15.6         82      1  
vw pickup                         24.6         82      2  
dodge rampage                     11.6         82      1  
ford ranger                       18.6         82      1  
chevy s-10                        19.4         82      1  

[398 rows x 8 columns]

In [7]:
#填補遺失值
mean_hp = pd.to_numeric(df['horsepower'], errors='coerce').mean()
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce').fillna(mean_hp)
print(df.isnull().sum())

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
dtype: int64


In [8]:
#model
for i in df.columns:
    y  = df[i]
    x = df.drop(i, axis=1)
    model = LinearRegression().fit(x, y)
    print(i)
    print("R squared : ", model.score(x, y))
    print("VIF : ", 1/(1-model.score(x, y)))
    print("---------------------------------")

mpg
R squared :  0.8209019153826466
VIF :  5.583532633174272
---------------------------------
cylinders
R squared :  0.9069092906118217
VIF :  10.742210544664632
---------------------------------
displacement
R squared :  0.9548634015164353
VIF :  22.154970325557972
---------------------------------
horsepower
R squared :  0.8893232425691762
VIF :  9.035320723278588
---------------------------------
weight
R squared :  0.9257323912540218
VIF :  13.464820220890086
---------------------------------
acceleration
R squared :  0.6021641524225465
VIF :  2.5135995312873685
---------------------------------
model year
R squared :  0.4884429978631091
VIF :  1.9548163661581615
---------------------------------
origin
R squared :  0.4602784832082043
VIF :  1.8528073624786807
---------------------------------


結論：觀察所有變數，其中cylinders, displacement, weight的VIF大於10，代表這些變數與其他變數高度相關，存在共線性問題。共線性問題會使Var(Bj)增加，而導致t-ratio test傾向不顯著結果，影響模型的結論。可以透過刪除displacement（最高的VIF），重建模型，重複此步驟，直到所有的VIF都在10以下。